# P53 - DMS Analyse
#### von Frido Petersen, Dario Prifti, Maximilian Fidlin und Enno Schäfer

## Grobe Struktur:
1) Data Cleanup: wir hatten diese Daten, das und das haben wir damit gemacht, so und so sehen unsere Daten jetzt aus
    (i) NA - remove
    (ii) Z - Transformierung
    (iii) Normalisierung
` `
` `
2) Data Exploration
    (i) Distanzmatrix
    (ii) Clustering
    (iii) weitere Clustering-Methoden
    (iv) Zusätzliche AS Daten
` `
` `
3) T-tests

## Data cleanup
 *Welche Funktionen habt ihr benutzt und warum (also warum haben wir ne z-transformation gemacht und nicht einfach direkt ne Normalisierung der Werte auf einen bestimmten Zahlenbereich?) Schreibt mal bitte auch eure Funktionen mit rein (also euren Code) und all die Sachen, die ihr außerhalb davon herausgefunden habt (also was ihr so rumgegoogelt und herausgefunden habt)*

Frido:
Datensatz in handlicheres Format bringen mit df_transform:

def df_transform(frame: pd.DataFrame) -> pd.DataFrame:
    """df wird in Format WT_Sequenz-Mutierte_AS umgewandelt"""
    frame['position_mut'] = frame.mutant.str.slice(start=1, stop=-1).astype(int)
    frame['AS_old'] = frame.mutant.str.get(0)
    frame['AS_new'] = frame.mutant.str.get(-1)
    pivoted_frame = frame.pivot(index='AS_new', columns=['position_mut', 'AS_old'], values='DMS_score')
    return pivoted_frame
DF ist dadurch so angeordnet, dass die obere Zeile die WT-Seq. angibt und dann in den Spalten jeweils die DMS-Scores an der Position für die Mutationen mit den verschiedenen Basen stehen.
-> Sorgt für NA an den Stellen, bei denen kein Austausch durchgeführt wurde
Sorgt dafür, dass wir nachfolgende Berechnungen und Analysen einfacher mit dem Datensatz durchführen können.

na_remove Funktion
def rmv_na(df: pd.DataFrame) -> pd.DataFrame:
    """Ändert die Werte eines df von NaN zu 0, wenn mutierte AS der WT AS entspricht"""
    for col in df.columns:
        for row in df.index:
            col_str = str(col)
            row_str = str(row)
            letters_row = ''.join(filter(str.isalpha, row_str))
            letters_col = ''.join(filter(str.isalpha, col_str))

            if letters_row == letters_col and pd.isna(df.loc[row, col]):
                df.loc[row, col] = 0

    return df

-> Entfernt die NA bei denen der Austausch nicht durchgeführt wurde, weil die AS der ursprünglichen AS entsprechen würde und ersetzt sie mit Null, da keine Veränderung in der Fitness zu erwarten

Dario:

Max: Wir haben probiert Patientendaten zu bekommen, aber wir haben die nicht bekommen

## Data exploration
*Was sind die Ergebnisse, auf die ihr gekommen seid? Also code mit einfügen, um dann plotten zu können*

Frido:
### Bearbeiten des AS-Datensatz
aa = pd.read_csv('../DMS_data/aminoacids.csv')

#Entfernen von Hydroxyproline und Pyroglutamat
aa_nat = aa.drop(index=[12, 18])

#Speichern der "Letter"-Spalte für Benennung der neuen Spalten
labels_column = 'Letter'

#Entfernen aller nicht-numerischen Spalten und pKx3, da unvollständig und Molecular Weight, da im Protein nicht relevant
aa_rmv = aa_nat.drop(['Name', 'Abbr','Letter', 'Molecular Formula','Molecular Weight', 'Residue Formula', 'pKx3'], axis=1)

#z-transformationen, damit unterschiedliche Messwerte nicht unterschiedlich stark ins Gewicht fallen
aa_zscore = aa_rmv.apply(stats.zscore)

#Berechnen der Distanzen der AA zueinander
aa_distances = euclidean_distances(aa_zscore.values)

#Umwandeln zurück in einen DF zur besseren Übersicht
aa_df = pd.DataFrame(aa_distances, index=aa_nat[labels_column], columns=aa_nat[labels_column])

### Distanzmatrix (DMS-Scores) -> Vllt. erklärst du vorher noch die Mean-Geschichte @Enno

from sklearn.metrics.pairwise import euclidean_distances

# Speichern der "AS_new"-Spalte für Benennung der neuen Spalten
labels_column = 'AS_new'

# Transpose the DataFrame
mean_substitutions_t = mean_substitutions.T

# Berechnen der Distanzen der AA zueinander
dms_distances = euclidean_distances(mean_substitutions_t.values)

# Umwandeln zurück in einen DF zur besseren Übersicht
dms_distances_df = pd.DataFrame(dms_distances, index=mean_substitutions_t.index, columns=mean_substitutions_t.index)

# Print the first 20 rows of mean_substitutions
dms_distances_df.head(20)

-> Gibt uns die Abstände der AS zueinander basierend  auf den DMS-Scores bei Austausch von AS. Bin mir leider nicht sicher ob die Abstände der neuen oder alten AS zueinander berechnet werden. Und check auch nicht ganz wieso das nur mit der transponierten Matrix funktioniert


Dario:

Max:

Enno: